<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#isnull" data-toc-modified-id="isnull-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>isnull</a></span><ul class="toc-item"><li><span><a href="#na" data-toc-modified-id="na-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>na</a></span></li><li><span><a href="#outer" data-toc-modified-id="outer-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>outer</a></span></li></ul></li></ul></div>

In [1]:
import os
from distutils.sysconfig import get_python_lib,get_python_inc,get_python_version

In [2]:
os.environ['SPARK_HOME']=f'{get_python_lib()}{os.sep}pyspark'
os.environ['PYSPARK_PYTHON']=f'{get_python_lib()}{os.sep}pyspark'.split('Lib')[0]+'python.exe'

In [3]:
from pyspark.sql import SparkSession
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types     import StructType, \
     StructField, FloatType, \
     IntegerType, StringType

In [4]:
spark = SparkSession \
    .builder \
    .appName("demo") \
    .config("spark.sql.warehouse.dir", 'F:\\FormationPython\\spark\\warehouse') \
    .enableHiveSupport() \
    .getOrCreate()

In [5]:
spark

In [6]:
from pyspark.sql.functions import *
from pyspark.sql.types     import StructType, \
     StructField, FloatType, \
     IntegerType, StringType

meteoDataFrame  = spark.read.format('csv')\
    .option('sep',';')\
    .option('header','true')\
    .option('nullValue','mq')\
    .option('inferSchema', 'true')\
    .load('./donnees/meteo')\
    .cache()

schema = StructType([
        StructField('Id'           , StringType() , True),
        StructField('ville'        , StringType() , True),
        StructField('latitude'     , FloatType() , True),
        StructField('longitude'    , FloatType() , True),
        StructField('altitude'     , IntegerType() , True)])

villes  = spark.read.format('csv')   \
      .option('sep',';')                \
      .option('mergeSchema', 'true')    \
      .option('header','true')          \
      .schema(schema)                   \
      .load('./donnees/postesSynop.csv')  \
      .cache()

@udf("string")
def formatVille(ville):
    if ville in ['CLERMONT-FD','MONT-DE-MARSAN',
                                   'ST-PIERRE','ST-BARTHELEMY METEO'] :
        return ville.title()
    else :
        if ville.find('-') != -1 :
            return ville[0:ville.find('-')].title()
        else:
            return ville.title()

villesT  = villes.select(
                col('Id').alias('id'),
                formatVille('ville').alias('ville'),
               'latitude',
               'longitude',
               'altitude')


meteo = meteoDataFrame.select(
                 col('numer_sta'),
                 col('date')[0:4].cast('int') ,
                 col('date')[5:2].cast('int'),
                 col('date')[7:2].cast('int'),
                 col('date')[5:4],
                 round(col('t') - 273.15,2),
                 col('u') / 100 ,
                 col('vv') / 1000 ,
                 col('pres') / 1000,
                 coalesce( col('rr3'),
                           col('rr24')/8,
                           col('rr12')/4,
                           col('rr6')/2,
                           col('rr1')*3  ) )\
             .toDF('id','annee','mois','jour','mois_jour','temperature',
                   'humidite','visibilite','pression','precipitations')\
             .cache()

meteo.select('annee','mois','jour','temperature','humidite',
             'visibilite','pression').toPandas().head(3)

,annee,mois,jour,temperature,humidite,visibilite,pression
0,2023,8,1,15.4,0.97,20.00,99.46
1,2023,8,1,15.6,0.95,51.23,99.63
2,2023,8,1,16.3,0.92,12.00,100.43


In [7]:
villesT.toPandas().head(3)

,id,ville,latitude,longitude,altitude
0,07005,Abbeville,50.136002,1.834000,69
1,07015,Lille,50.570000,3.097500,47
2,07020,Pte De La Hague,49.725166,-1.939833,6


# isnull

In [8]:
meteo.where('id < 8000').count()
description = meteo.where('id < 8000')\
     .select('annee','temperature', 'humidite',
             'visibilite', 'pression','precipitations')\
     .describe()\
     .select([c if c == 'summary'
                else round(c,2).alias(c)
                for c in
                    ['summary','annee','temperature', 'humidite',
                   'visibilite', 'pression','precipitations']])\
     .toPandas()

In [9]:
meteo.where('id < 8000').where(meteo['temperature'].isNull()).count()

9156

In [10]:
meteo.where('id < 8000').where(meteo['humidite'].isNull()).count()

9629

In [11]:
meteo.where('id < 8000').where(meteo['visibilite'].isNull()).count()

40783

In [12]:
meteo.where('id < 8000').where(meteo['pression'].isNull()).count()

8207

In [13]:
meteo.where('id < 8000').where(meteo['precipitations'].isNull()).count()

9961

In [14]:
meteo.where('id < 8000').toPandas().isna().sum()

id                    0
annee                 0
mois                  0
jour                  0
mois_jour             0
temperature        9156
humidite           9629
visibilite        40783
pression           8207
precipitations     9961
dtype: int64

In [15]:
meteoDataFrame.toPandas().isna().sum()

Py4JJavaError: An error occurred while calling o38.collectToPython.
: java.lang.OutOfMemoryError: GC overhead limit exceeded
	at org.apache.spark.sql.execution.SparkPlan$$anon$1._next(SparkPlan.scala:417)
	at org.apache.spark.sql.execution.SparkPlan$$anon$1.getNext(SparkPlan.scala:426)
	at org.apache.spark.sql.execution.SparkPlan$$anon$1.getNext(SparkPlan.scala:412)
	at org.apache.spark.util.NextIterator.hasNext(NextIterator.scala:73)
	at scala.collection.Iterator.foreach(Iterator.scala:943)
	at scala.collection.Iterator.foreach$(Iterator.scala:943)
	at org.apache.spark.util.NextIterator.foreach(NextIterator.scala:21)
	at org.apache.spark.sql.execution.SparkPlan.$anonfun$executeCollect$1(SparkPlan.scala:449)
	at org.apache.spark.sql.execution.SparkPlan.$anonfun$executeCollect$1$adapted(SparkPlan.scala:448)
	at org.apache.spark.sql.execution.SparkPlan$$Lambda$3805/1345908592.apply(Unknown Source)
	at scala.collection.IndexedSeqOptimized.foreach(IndexedSeqOptimized.scala:36)


In [16]:
# from databricks import koalas as ks
meteoDataFrame.to_koalas().isna().sum()

C:\anaconda3\envs\spark3\lib\site-packages\pyspark\pandas\__init__.py:50: UserWarning: 'PYARROW_IGNORE_TIMEZONE' environment variable was not set. It is required to set this environment variable to '1' in both driver and executor sides if you use pyarrow>=2.0.0. pandas-on-Spark will set it for you but it does not work if there is a Spark context already launched.
  warnings.warn(


numer_sta         0
date              0
pmer          55169
tend          27661
cod_tend      43210
dd            11419
ff            11196
t             10458
td            16598
u             15929
vv           224892
ww           244668
w1           858370
w2           908980
n            596121
nbas         334327
hbas         520501
cl           870204
cm           900910
ch           914327
pres          18290
niv_bar      992931
geop         969991
tend24        60848
tn12         798632
tn24         846312
tx12         801555
tx24         845194
tminsol      389610
sw           998251
tw           998251
raf10        229406
rafper       195191
per          192306
etat_sol     545194
ht_neige     435064
ssfrai       949989
perssfrai    949989
rr1           42031
rr3           42512
rr6           63953
rr12          76768
rr24          90402
phenspe1     998251
phenspe2     998251
phenspe3     998251
phenspe4     998251
nnuage1      523546
ctype1       876338
hnuage1      544514


In [17]:
meteo.where('id < 8000')\
     .where(meteo['temperature'].isNotNull())\
     .where(meteo['humidite'].isNotNull() )\
     .where(meteo['visibilite'].isNotNull() )\
     .where(meteo['pression'].isNotNull() )\
     .count()

664574

## na

In [18]:
meteo.where('id < 8000')\
     .na.fill(0 ,["precipitations"])\
     .na.drop()\
     .count()

664574

In [19]:
meteo = meteoDataFrame.select(
                 col('numer_sta'),
                 col('date')[0:4].cast('int') ,
                 col('date')[5:2].cast('int'),
                 col('date')[7:2].cast('int'),
                 col('date')[5:4],
                 round(col('t') - 273.15,2),
                 col('u') / 100 ,
                 col('vv') / 1000 ,
                 col('pres') / 1000,
                 col('rr1')*3,
                 col('rr3'),
                 col('rr6')/2,
                 col('rr12')/4,
                 col('rr24')/8)\
                   .toDF('id','annee','mois','jour','mois_jour','temperature',
                   'humidite','visibilite','pression',
                   'precipitations1','precipitations3','precipitations6',
                   'precipitations12','precipitations24')\
             .cache()

In [20]:
meteo.where('id < 8000')\
       .select('precipitations1','precipitations3','precipitations6',
             'precipitations12','precipitations24','temperature')\
       .toDF('prec1','prec3','prec6','prec12','prec24','temp')\
       .describe()\
       .select([c if c == 'summary'
                else round(c,2).alias(c)
                for c in['prec1','prec3','prec6',
                          'prec12','prec24','temp']])\
      .toPandas()

,prec1,prec3,prec6,prec12,prec24,temp
0,693279.00,696238.00,686342.00,682071.00,674640.00,700321.00
1,0.22,0.23,0.23,0.24,0.24,13.57
2,1.63,1.21,0.99,0.79,0.62,7.51
3,-0.30,-0.10,-0.05,-0.03,-0.01,-16.60
4,184.50,111.80,57.50,28.85,23.94,42.30


In [21]:
meteo.select( coalesce('precipitations3','precipitations24',
                       'precipitations12','precipitations6',
                       'precipitations1').alias('precipitations')).toPandas().head(20)

,precipitations
0,0.4
1,0.6
2,0.0
3,2.0
4,13.4
5,2.4
6,-0.1
7,0.2
8,0.6
9,3.0


In [22]:
meteo.where('id < 8000')\
     .select( coalesce('precipitations24','precipitations12',
                       'precipitations6','precipitations3',
                       'precipitations1').alias('precipitationsH')
            ).describe().toPandas()

,summary,precipitationsH
0,count,699516
1,mean,0.23698056942225346
2,stddev,0.6483763868230633
3,min,-0.30000000000000004
4,max,30.0


In [23]:
meteo.to_koalas().isna().sum()

id                       0
annee                    0
mois                     0
jour                     0
mois_jour                0
temperature          10458
humidite             15929
visibilite          224892
pression             18290
precipitations1      42031
precipitations3      42512
precipitations6      63953
precipitations12     76768
precipitations24     90402
dtype: int64

In [24]:
meteo.where('id < 8000').count()

709477